# Async Programming Basics

## Introduction

Async programming is essential for AI engineering because:
- **API calls are I/O-bound**: Waiting for network responses
- **LLMs take time**: GPT-4 responses can take 5-30 seconds
- **Batch processing**: Process multiple prompts efficiently
- **Real-world scale**: Production systems handle many concurrent requests

**Key insight:** While one API call waits for a response, your program can start other calls.

## Learning Objectives

By the end of this notebook, you will:
1. Understand the difference between sync and async code
2. Use `async`/`await` syntax correctly
3. Run async code in Jupyter notebooks
4. Understand the event loop
5. Know when to use async vs sync code

## 1. Synchronous vs Asynchronous

### Synchronous (Traditional) Code

Synchronous code executes one line at a time. Each operation must complete before the next begins.

In [ ]:
import time

def fetch_data_sync(url: str, delay: float) -> str:
    """Simulate a synchronous API call."""
    print(f"Starting request to {url}...")
    time.sleep(delay)  # Simulate network delay
    print(f"Completed request to {url}")
    return f"Data from {url}"

# Measure time for sequential execution
start = time.time()

result1 = fetch_data_sync("api.example.com/1", 2)
result2 = fetch_data_sync("api.example.com/2", 2)
result3 = fetch_data_sync("api.example.com/3", 2)

elapsed = time.time() - start
print(f"\nTotal time: {elapsed:.2f} seconds")
print(f"Results: {result1}, {result2}, {result3}")

**Output explanation:**
- Each request waits for the previous to complete
- Total time: ~6 seconds (2 + 2 + 2)
- CPU is idle while waiting for "network"

This is **wasteful** for I/O-bound operations like API calls.

### Asynchronous Code

Async code can start multiple operations and wait for all to complete.

In [ ]:
import asyncio

async def fetch_data_async(url: str, delay: float) -> str:
    """Simulate an asynchronous API call."""
    print(f"Starting request to {url}...")
    await asyncio.sleep(delay)  # Simulate network delay (non-blocking)
    print(f"Completed request to {url}")
    return f"Data from {url}"

# Measure time for concurrent execution
start = time.time()

# Create tasks for concurrent execution
results = await asyncio.gather(
    fetch_data_async("api.example.com/1", 2),
    fetch_data_async("api.example.com/2", 2),
    fetch_data_async("api.example.com/3", 2)
)

elapsed = time.time() - start
print(f"\nTotal time: {elapsed:.2f} seconds")
print(f"Results: {results}")

**Output explanation:**
- All requests start at the same time
- Total time: ~2 seconds (all run concurrently)
- **3x faster** than synchronous version

This is the **power of async** for I/O-bound operations.

## 2. Async/Await Syntax

### The `async` Keyword

- Defines a **coroutine** (async function)
- Can be paused and resumed
- Returns a coroutine object (not the result directly)

In [ ]:
async def simple_async_function():
    """A simple async function."""
    return "Hello from async!"

# Calling async function returns a coroutine
result = simple_async_function()
print(f"Type: {type(result)}")
print(f"Result: {result}")

# To get the actual result, use await
actual_result = await simple_async_function()
print(f"Actual result: {actual_result}")

### The `await` Keyword

- **Pauses** the coroutine until the awaited operation completes
- Only works inside `async` functions
- Allows other code to run while waiting

In [ ]:
async def step_one():
    print("Step 1: Starting...")
    await asyncio.sleep(1)
    print("Step 1: Done!")
    return "Step 1 result"

async def step_two():
    print("Step 2: Starting...")
    await asyncio.sleep(1)
    print("Step 2: Done!")
    return "Step 2 result"

async def main():
    # Sequential: wait for each step
    print("=== Sequential ===")
    result1 = await step_one()
    result2 = await step_two()
    print(f"Results: {result1}, {result2}\n")
    
    # Concurrent: start both at once
    print("=== Concurrent ===")
    results = await asyncio.gather(step_one(), step_two())
    print(f"Results: {results}")

await main()

## 3. Common Async Patterns

### Pattern 1: asyncio.gather() - Run Multiple Tasks

Use when you want to run multiple async operations concurrently and collect all results.

In [ ]:
async def process_item(item_id: int) -> str:
    await asyncio.sleep(0.5)  # Simulate processing
    return f"Processed item {item_id}"

# Process 5 items concurrently
items = [1, 2, 3, 4, 5]

start = time.time()
results = await asyncio.gather(*[process_item(i) for i in items])
elapsed = time.time() - start

print(f"Results: {results}")
print(f"Time: {elapsed:.2f}s (would be {0.5 * len(items):.2f}s if sequential)")

### Pattern 2: asyncio.create_task() - Fire and Forget

Use when you want to start a task without immediately waiting for it.

In [ ]:
async def background_task(name: str):
    print(f"{name}: Starting...")
    await asyncio.sleep(2)
    print(f"{name}: Done!")
    return f"{name} result"

async def main():
    # Start tasks
    task1 = asyncio.create_task(background_task("Task A"))
    task2 = asyncio.create_task(background_task("Task B"))
    
    print("Tasks started, doing other work...")
    await asyncio.sleep(0.5)
    print("Other work done, now waiting for tasks...")
    
    # Wait for tasks to complete
    result1 = await task1
    result2 = await task2
    
    print(f"Results: {result1}, {result2}")

await main()

### Pattern 3: asyncio.wait_for() - Timeout

Use when you want to limit how long you wait for an operation.

In [ ]:
async def slow_operation():
    await asyncio.sleep(5)  # Takes 5 seconds
    return "Success!"

try:
    # Timeout after 2 seconds
    result = await asyncio.wait_for(slow_operation(), timeout=2.0)
    print(f"Result: {result}")
except asyncio.TimeoutError:
    print("Operation timed out!")

## 4. The Event Loop

The **event loop** is the engine that runs async code.

### How it works:
1. You submit coroutines to the event loop
2. Event loop runs them
3. When a coroutine hits `await`, it pauses
4. Event loop switches to another coroutine
5. When the awaited operation completes, the coroutine resumes

### In Jupyter Notebooks

Jupyter has a built-in event loop, so you can use `await` directly:

In [ ]:
# In Jupyter, this works directly:
result = await asyncio.sleep(1, result="Done!")
print(result)

### In Regular Python Scripts

In regular scripts, you need to explicitly run the event loop:

In [ ]:
# This is how you'd run async code in a .py file:

async def main():
    result = await asyncio.sleep(1, result="Done!")
    print(result)

# Two ways to run:
# 1. Python 3.7+ (recommended)
# asyncio.run(main())

# 2. Manual event loop (older Python)
# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())

print("In Jupyter, we can just use await directly!")
await main()

## 5. When to Use Async

### ✅ Use Async For:

- **API calls**: Waiting for HTTP responses
- **Database queries**: Waiting for database
- **File I/O**: Reading/writing large files
- **Network operations**: Any network communication
- **Multiple I/O-bound tasks**: Running many I/O operations

### ❌ Don't Use Async For:

- **CPU-bound tasks**: Heavy computation (use multiprocessing instead)
- **Simple scripts**: If you're only making 1-2 API calls
- **Synchronous libraries**: If the library doesn't support async

### AI Engineering Use Cases:

1. **Batch prompt processing**: Process 100 prompts concurrently
2. **Multiple model calls**: Query GPT-4 and Claude simultaneously
3. **RAG systems**: Fetch documents while generating response
4. **Agent systems**: Run multiple agents in parallel
5. **Real-time streaming**: Handle WebSocket connections

## 6. Common Pitfalls

### Pitfall 1: Forgetting `await`

In [ ]:
async def get_data():
    await asyncio.sleep(1)
    return "data"

# ❌ Wrong: Returns coroutine object
result_wrong = get_data()
print(f"Wrong: {result_wrong}")

# ✅ Correct: Use await
result_correct = await get_data()
print(f"Correct: {result_correct}")

### Pitfall 2: Using `time.sleep()` instead of `asyncio.sleep()`

In [ ]:
async def bad_async():
    time.sleep(1)  # ❌ Blocks the entire event loop!
    return "bad"

async def good_async():
    await asyncio.sleep(1)  # ✅ Allows other tasks to run
    return "good"

# This will take 2 seconds (bad blocks everything)
start = time.time()
results = await asyncio.gather(bad_async(), good_async())
print(f"Time: {time.time() - start:.2f}s - Should be ~1s but bad_async blocks!")

### Pitfall 3: Not using `asyncio.gather()` for concurrent execution

In [ ]:
async def fetch(n):
    await asyncio.sleep(1)
    return n

# ❌ Sequential: Each await blocks
start = time.time()
result1 = await fetch(1)
result2 = await fetch(2)
result3 = await fetch(3)
print(f"Sequential: {time.time() - start:.2f}s")

# ✅ Concurrent: All run at once
start = time.time()
results = await asyncio.gather(fetch(1), fetch(2), fetch(3))
print(f"Concurrent: {time.time() - start:.2f}s")

## 7. Practice Exercise

Create an async function that simulates checking the status of multiple AI models:

In [ ]:
async def check_model_status(model_name: str) -> dict:
    """
    Simulate checking if an AI model is available.
    
    Args:
        model_name: Name of the model to check
        
    Returns:
        Dictionary with model status
    """
    # TODO: Implement this function
    # - Simulate a 1-second API call with asyncio.sleep
    # - Return a dict with {"model": model_name, "status": "available"}
    pass

# Test your function
models = ["gpt-4", "claude-3", "llama-2", "mistral-7b"]

start = time.time()
# TODO: Use asyncio.gather to check all models concurrently
# results = await asyncio.gather(...)
elapsed = time.time() - start

print(f"Checked {len(models)} models in {elapsed:.2f}s")
# Should be ~1 second, not 4 seconds!
# for result in results:
#     print(result)

## Summary

### Key Takeaways:

1. **Async is for I/O-bound operations** like API calls
2. **`async def` defines a coroutine**, `await` pauses it
3. **`asyncio.gather()` runs multiple tasks concurrently**
4. **In Jupyter, use `await` directly** (event loop already running)
5. **Always use `asyncio.sleep()` not `time.sleep()` in async code**

### Next Steps:

- **Notebook 2**: Concurrent API calls with real examples
- **Notebook 3**: Advanced async patterns for AI applications
- **Exercise**: Build concurrent OpenAI API client

### Resources:

- [Official asyncio docs](https://docs.python.org/3/library/asyncio.html)
- [Real Python async tutorial](https://realpython.com/async-io-python/)
- [asyncio cheatsheet](https://www.pythonsheets.com/notes/python-asyncio.html)